# Kindlmann Color Map

This iPython notebook contains the script required to derive what is often known as the Kindlmann color map, so named because its first known design is in [a paper by Kindlmann, Reinhard, and Creem](http://www.cs.utah.edu/~gk/papers/vis02/), although this derivation takes inspiration from other sources. The map is basically the rainbow color map with the luminance adjusted such that it monotonically changes, making it much more perceptually viable.

This code relies on the [python-colormath](http://python-colormath.readthedocs.org/en/latest/index.html) module. See [its documentation](http://python-colormath.readthedocs.org/en/latest/index.html) for information such as installation instructions. (It can be installed with either pip or macports.)

In [1]:
from colormath.color_objects import *
from colormath.color_conversions import convert_color

Mostly because it's habit, I am also using [pandas](http://pandas.pydata.org/) dataframes to organize the data. (Pandas can be installed with macports.)

In [2]:
import pandas
import numpy

We will also be using [toyplot](https://toyplot.readthedocs.org) for making visuals. See its documentation for installation instructions.

In [3]:
import toyplot
import toyplot.pdf

### Support Functions

The original Kindlmann paper created their color map by having a human user adjust the brightness to a prescribed luminance. (The color map was really proposed as a use case for a technique that allows humans to match colors.) Personally, I think this technique is overkill. First of all, no one is going to calibrate to their display. (*I* don't even want to do it one time just to get initial values.) Second, any change in the display will invalidate the calibration anyway.

Instead, I am going to use a programmatic technique proposed in [a blog post by Matteo Niccoli](https://mycarta.wordpress.com/2012/12/06/the-rainbow-is-deadlong-live-the-rainbow-part-5-cie-lab-linear-l-rainbow/). The idea is to convert the RGB values to the perceptual CIELAB space, adjust the L (luminance) value in CIELAB, and then convert back to RGB.

To do this, we need a function that takes a hue value and adjusts its luminance. Of course, we need to convert to CIELAB and alter the L value. But then we also have to adjust the a and b values so that the color is back in the representable color gamut. We do this with a binary search.

In [ ]:
def adjust_color(hue, scalar):
    '''Given a hue value (in degrees) and a scalar value between
    0 and 1, adjust the color to have a luminance proportional to
    the scalar. Returns an sRGBColor value.'''

### Original Color Map

The color map in the original Kindlmann paper follows the rainbow hues from purple through blue and green to red. We will use those hues first, but also derive a slightly alternate one below.

In [12]:
rgb = sRGBColor(0, 0, 1)
convert_color(rgb, HSVColor)

HSVColor(hsv_h=240.0,hsv_s=1.0,hsv_v=1.0)

In [11]:
start_hue = 300.0
end_hue = 0.0

We start by creating a "short" map with a minimal amount of control points. These control points are placed where the RGB interpolation bends, which every 60 degrees in HSV space. Create a table starting with the scalar values and the hue angle for each one.

In [17]:
data_short = pandas.DataFrame()
data_short['hue'] = numpy.arange(start_hue, end_hue-0.0001, -60.0)
data_short['scalar'] = numpy.linspace(0.0, 1.0, data['hue'].size)
data_short

,hue,scalar
0,300,0.0
1,240,0.2
2,180,0.4
3,120,0.6
4,60,0.8
5,0,1.0
